In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

In [2]:
examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝 ",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Can you make a emoji of {movie}?"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

few_shot_final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert who expresses the contents of the movie with 3 emoticons. If I ask a question in a sentence, just answer it in a sentence.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        few_shot_prompt,
        ("human", "{movie}"),
    ]
)

In [3]:
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = (
    RunnablePassthrough.assign(chat_history=load_memory) | few_shot_final_prompt | llm
)


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result)

In [4]:
invoke_chain("avatar")

content='🪐🌿🗡️'


In [5]:
invoke_chain("iron man")

content='🦾💥🔴'


In [6]:
invoke_chain("what is the first movie I asked was?")

content='Avatar'
